# Graphical User Interfaces

Object oriented programming and particularly inheritance is commonly used for creating GUIs. There are [a large number](https://wiki.python.org/moin/GuiProgramming) of different frameworks supporting building GUIs. The following are particularly relevant:

* [TkInter](https://docs.python.org/3.5/library/tkinter.html) - This is the official/default GUI framework
* [guidata](https://pythonhosted.org/guidata/) - A GUI framework for dataset display and editing
* [VTK](http://www.vtk.org/) - A GUI framework for data visualization
* [pyqtgraph](http://www.pyqtgraph.org/documentation/index.html) - A GUI framework for data visualization, easily installed with `conda install pyqtgraph`
* [matplotlib](http://matplotlib.org/) - As well as creating plots matplotlib can support interaction

## Matplotlib

For simple programs, displaying data and taking basic input, often a command line application will be much faster to implement than a GUI. The times when I have moved away from the command line it has been to interact with image data and plots. Here, matplotlib often works very well. Either it can be embedded in a larger application or it can be used directly.

There are a number of examples on the [matplotlib site](http://matplotlib.org/examples/index.html).

Here is a stripped down example of a GUI I recently used.

In [15]:
"""
Do a mouseclick somewhere, move the mouse to some destination, release
the button.  This class gives click- and release-events and also draws
a line or a box from the click-point to the actual mouseposition
(within the same axes) until the button is released.  Within the
method 'self.ignore()' it is checked wether the button from eventpress
and eventrelease are the same.

"""
from matplotlib.widgets import RectangleSelector
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook


def line_select_callback(eclick, erelease):
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    print("({0:.2f}, {1:.2f}) --> ({2:.2f}, {3:.2f})".format(x1, y1, x2, y2))
    print("The button you used were: {0} {1}".format(eclick.button, erelease.button))
    print('\n')

    
def toggle_selector(event):
    print (' Key pressed.')
    if event.key in ['Q', 'q'] and toggle_selector.RS.active:
        print ('RectangleSelector deactivated.')
        toggle_selector.RS.set_active(False)
    if event.key in ['A', 'a'] and not toggle_selector.RS.active:
        print ('RectangleSelector activated.')
        toggle_selector.RS.set_active(True)


        
image_file = cbook.get_sample_data('grace_hopper.png')
image = plt.imread(image_file)
fig, current_ax = plt.subplots()
plt.imshow(image)
toggle_selector.RS = RectangleSelector(current_ax, 
        line_select_callback,
        drawtype='box', useblit=True,
        button=[1,3], # don't use middle button
        minspanx=5, minspany=5,
        spancoords='pixels')
plt.connect('key_press_event', toggle_selector)
plt.show()


(116.84, 157.69) --> (376.82, 422.40)
The button you used were: 1 1




We are using a variety of objects here and our code still looks very procedural. For a simple task such as this we don't need to give much thought to how we structure our program. As GUI projects become more complex applying object oriented concepts becomes more attractive.

We can take the very simple GUI below and apply a, by now hopefully familiar, analysis approach.

![Simple GUI](simple_gui.png)

What are the components of this GUI? What are the __things__ that are involved?

Are any components similar to each other? Do they have a '__is a__' type relationship? 

Which components will interact with each other? Do they have a '__has a__' type relationship?

## TkInter

TkInter is widely used with plenty of documentation available but may prove somewhat limited for more data intensive applications.

* [Documentation from the standard library](https://docs.python.org/3/library/tk.html)
* [Further documentation from python.org](https://docs.python.org/3.5/library/tkinter.html)
* [TkDocs](http://www.tkdocs.com/index.html)

Let's look at a simple example from the documentation

In [16]:
import tkinter as tk

class Application(tk.Frame):
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.pack()
        self.createWidgets()

    def createWidgets(self):
        self.hi_there = tk.Button(self)
        self.hi_there["text"] = "Hello World\n(click me)"
        self.hi_there["command"] = self.say_hi
        self.hi_there.pack(side="top")

        self.QUIT = tk.Button(self, text="QUIT", fg="red",
                                            command=root.destroy)
        self.QUIT.pack(side="bottom")

    def say_hi(self):
        print("hi there, everyone!")

root = tk.Tk()
app = Application(master=root)
app.mainloop()

Although this works, it is visually unappealing. We can improve on this using styles and themes.

In [44]:
import tkinter as tk
from tkinter import ttk


class Application(ttk.Frame):
    def __init__(self, master=None):
        super().__init__(master, padding="3 3 12 12")
        self.grid(column=0, row=0, )
        self.createWidgets()
        self.master.title('Test')

    def createWidgets(self):
        self.hi_there = ttk.Button(self)
        self.hi_there["text"] = "Hello World\n(click me)"
        self.hi_there["command"] = self.say_hi

        self.QUIT = ttk.Button(self, text="QUIT", style='Alert.TButton', command=root.destroy)

        for child in self.winfo_children(): 
            child.grid_configure(padx=10, pady=10)

    def say_hi(self):
        print("hi there, everyone!")

        

root = tk.Tk()
app = Application(master=root)
s = ttk.Style()
s.configure('TButton', font='helvetica 24')
s.configure('Alert.TButton', foreground='red')
root.mainloop()

Our Application class is very similar to our previous code. By applying style attributes outside of the core components and logic we can keep the program straightforward and easy to modify.

In addition to manually modifying the appearance of different parts of our GUI we can also apply styles.

In [33]:
s.theme_names()

('winnative', 'clam', 'alt', 'default', 'classic', 'vista', 'xpnative')

In [34]:
root = tk.Tk()
app = Application(master=root)
s = ttk.Style()
s.theme_use('clam')
root.mainloop()

As our applications get more complicated we must give greater thought to the layout. The following example comes from the [TkDocs site](http://www.tkdocs.com/tutorial/firstexample.html).

Notice, we are not using a class to encapsulate our code here. 

In [46]:
import tkinter as tk
from tkinter import ttk

def calculate(*args):
    try:
        value = float(feet.get())
        meters.set(0.3048 * value)
    except ValueError:
        pass
    
root = tk.Tk()
root.title("Feet to Meters")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

feet = tk.StringVar()
meters = tk.StringVar()

feet_entry = ttk.Entry(mainframe, width=7, textvariable=feet)
feet_entry.grid(column=2, row=1, sticky=(W, E))

ttk.Label(mainframe, textvariable=meters).grid(column=2, row=2, sticky=(W, E))
ttk.Button(mainframe, text="Calculate", command=calculate).grid(column=3, row=3, sticky=W)

ttk.Label(mainframe, text="feet").grid(column=3, row=1, sticky=W)
ttk.Label(mainframe, text="is equivalent to").grid(column=1, row=2, sticky=E)
ttk.Label(mainframe, text="meters").grid(column=3, row=2, sticky=W)

for child in mainframe.winfo_children(): 
    child.grid_configure(padx=5, pady=5)

feet_entry.focus()
root.bind('<Return>', calculate)

root.mainloop()

## A practical (and complex) example

Recently, I built a small GUI to label a collection of images. The goal was to display and interact (zoom and pan) with images to establish, view and edit masks of image regions. At later stages in the project manually drawn masks and masks from a machine learning model were displayed simultaneously.

The images were whole slide scans of histology samples and were very large. Loading the entire image into memory was not possible and much of the complexity in the project was due to this limitation.

As I was the only person using the GUI a number of quirks remain I would feel compelled to resolve if distributing the application to collaborators or more widely. Despite this, the project demonstrates many of the steps and approaches outlined for the simpler programs above.